In [ ]:
# ваша отправная точка — эта страница: https://www.hse.ru/org/persons/?ltr=%D0%90;udept=22726
# это список преподавателей и сотрудников НИУ ВШЭ, чьи фамилии начинаются на букву А
# вам нужно сначала собрать с каждого из них имя, кем работает и где, контакты (телефон, имейл), а также ссылку на страничку
# вторым туром вы проходитесь по каждой ссылке на преподавателя и достаете строчку, в которой говорится, в каком году человек начал работать в НИУ ВШЭ


#на примере первого человека — Абакумова Ани Васпуровна
# после первогой части работы у вас должно быть собрано [Абакумова Ани Васпуровна, Тьютор, 	Школа иностранных языков, +7 (915) 012-10-89, aabakumova@hse.ru, https://www.hse.ru/org/persons/224839542]
# а после второго к этому списку добавляется "Начала работать в НИУ ВШЭ в 2018 году."

# то есть в итоге у вас на каждого человека получится такая строчка 
# [Абакумова Ани Васпуровна, Тьютор, Школа иностранных языков, +7 (915) 012-10-89, aabakumova@hse.ru, https://www.hse.ru/org/persons/224839542, "Начала работать в НИУ ВШЭ в 2018 году."]

#сдать вам нужно будет табличку в csv и тетрадку с кодом, запустив который, я получу это табличку


In [1]:
#импортируйте библиотеки

import requests #чтобы получать код страница сайта
from bs4 import BeautifulSoup #чтобы "ходить" по тэгам,доставать нужную нам информацию
from bs4.element import NavigableString
import pandas as pd #для работы с таблицами и файлами
from time import sleep
import json

In [11]:
data = []

url = 'https://www.hse.ru/org/persons/?ltr=%D0%90;udept=22726'
r = requests.get(url) #делаем запрос к сайту
sleep(0.5)
soup = BeautifulSoup(r.text, 'lxml', from_encoding='utf-8')
persons = soup.findAll('div', class_='post__content_person')
for p in persons:
    anchor = p.find('div', class_='content__inner_foot1').find('a')
    name = anchor.text.strip()
    link = 'https://www.hse.ru/'+anchor.get('href')
    c = p.find('p', class_='with-indent7').find('span').contents
    profession = c[0].strip()
    
    department = ''
    for element in c[1:]:
        if type(element) is NavigableString:
            department = department+element.strip()
        else: 
            department = department+element.text.strip()
            
    phone = ''
    try:
        phone = p.find('div', class_='l-extra').find('span').text
    except:
        pass
    
    emails = []
    try:
        emails = p.find('div', class_='l-extra').findAll('a', class_='link')
    except:
        pass
    
    many_emails = ''
    for email in emails:
        symbols = json.loads(email.get('data-at'))
        many_emails = many_emails + ''.join(symbols).replace('-at-', "@") + ','
       
    data.append([name, profession, department.replace("\t",'').replace("\n",''), phone, many_emails[:-1], link])
            
for person in data:
    r = requests.get(person[-1]) 
    soup = BeautifulSoup(r.text, 'lxml', from_encoding='utf-8')
    
    person[-1] = soup.find('ul', class_='person-employment-addition').find('li').text
    
    
import csv 

with open('Tutors_list.csv', 'w') as csv_file:
    writer = csv.writer(csv_file, delimiter=';')

    for row in data:
        try:
            writer.writerow(row)
        except:
            pass
        




    

    

    